# 4D Gaussian Splatting from Multi-Camera Video

This notebook trains a 4D Gaussian Splatting model from your synchronized multi-camera footage.

**Prerequisites:**
- Upload `dataset.zip` containing your extracted frames
- Structure: `dataset/cam00/frame_00001.jpg, ...`

**Runtime:** Set to GPU (T4 or A100)

## 1. Setup Environment

In [ ]:
# Clone 4D Gaussians repository
!git clone https://github.com/hustvl/4DGaussians.git --recursive
%cd 4DGaussians

In [ ]:
# Install dependencies
!pip install -q plyfile tqdm scipy opencv-python imageio
!pip install -e submodules/depth-diff-gaussian-rasterization
!pip install -e submodules/simple-knn

# Verify GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install COLMAP for camera pose estimation
!apt-get update -qq
!apt-get install -qq colmap
print("\n✅ COLMAP installed")
!colmap -h | head -3

## 2. Upload Your Dataset

In [ ]:
# Upload dataset.zip
from google.colab import files
print("Upload your dataset.zip file:")
uploaded = files.upload()

In [ ]:
# Extract dataset
import os
import shutil

# Find uploaded zip
zip_file = [f for f in uploaded.keys() if f.endswith('.zip')][0]
print(f"Extracting {zip_file}...")

!mkdir -p data/multipleview
!unzip -q "{zip_file}" -d data/multipleview/

# Find and rename dataset folder
extracted = os.listdir('data/multipleview/')
if 'dataset' in extracted:
    !mv data/multipleview/dataset data/multipleview/my_scene
elif extracted and extracted[0] != 'my_scene':
    os.rename(f'data/multipleview/{extracted[0]}', 'data/multipleview/my_scene')

print("\n📁 Dataset structure:")
!ls -la data/multipleview/my_scene/

# Count frames per camera
print("\n📊 Frames per camera:")
for cam in sorted(os.listdir('data/multipleview/my_scene')):
    cam_path = f'data/multipleview/my_scene/{cam}'
    if os.path.isdir(cam_path):
        frames = len([f for f in os.listdir(cam_path) if f.endswith('.jpg')])
        print(f"  {cam}: {frames} frames")

## 3. Run COLMAP (Camera Pose Estimation)

This step estimates camera positions and creates an initial 3D point cloud.

⏱️ **Time:** 10-30 minutes depending on frame count

In [ ]:
# Run COLMAP pipeline
!bash multipleviewprogress.sh my_scene

In [ ]:
# Verify COLMAP output
import os

sparse_dir = 'data/multipleview/my_scene/sparse_'
point_cloud = 'data/multipleview/my_scene/points3D_multipleview.ply'

print("Checking COLMAP outputs...")

if os.path.exists(sparse_dir):
    print(f"✅ Camera poses: {sparse_dir}/")
    !ls -la {sparse_dir}/
else:
    print(f"❌ Missing: {sparse_dir}")

if os.path.exists(point_cloud):
    size_mb = os.path.getsize(point_cloud) / 1e6
    print(f"\n✅ Point cloud: {point_cloud} ({size_mb:.1f} MB)")
else:
    print(f"\n❌ Missing: {point_cloud}")

## 4. Configure Training

In [ ]:
# Create training configuration
config = '''
ModelParams = dict(
    name = "my_scene",
    deform_type = 'deform',
    is_blender = False,
    hyper = False,
    init_point_cloud = "points3D_multipleview.ply",
)

OptimizationParams = dict(
    coarse_iterations = 3000,
    iterations = 10000,
    position_lr_init = 0.00016,
    position_lr_final = 0.0000016,
    deformation_lr_init = 0.00016,
    deformation_lr_final = 0.000016,
    densify_until_iter = 5000,
    densify_grad_threshold = 0.0002,
    opacity_reset_interval = 3000,
    prune_interval = 100,
    batch_size = 1,
)

ModelHiddenParams = dict(
    net_width = 64,
    timebase_pe = 4,
    defor_depth = 1,
    posebase_pe = 10,
    scale_rotation_activation = "sigmoid",
    multires = [1, 2, 4, 8],
)
'''

os.makedirs('arguments/multipleview', exist_ok=True)
with open('arguments/multipleview/my_scene.py', 'w') as f:
    f.write(config)

print("✅ Config saved to arguments/multipleview/my_scene.py")

## 5. Train 4D Gaussian Splatting

⏱️ **Time:** 
- T4 GPU: ~30-60 minutes
- A100 GPU: ~15-20 minutes

In [ ]:
# Start training
!python train.py \
    -s data/multipleview/my_scene \
    --port 6017 \
    --expname "multipleview/my_scene" \
    --configs arguments/multipleview/my_scene.py

## 6. Render Results

In [ ]:
# Render novel views
!python render.py \
    --model_path output/multipleview/my_scene \
    --configs arguments/multipleview/my_scene.py \
    --skip_train

In [ ]:
# Create video from rendered frames
import imageio
import glob
from pathlib import Path

# Find render directory
render_dirs = list(Path('output/multipleview/my_scene').glob('**/renders'))
if not render_dirs:
    render_dirs = list(Path('output/multipleview/my_scene').glob('**/*.png'))
    if render_dirs:
        render_dir = render_dirs[0].parent
    else:
        print("No renders found")
        render_dir = None
else:
    render_dir = render_dirs[0]

if render_dir:
    frames = sorted(glob.glob(str(render_dir / '*.png')))
    if frames:
        print(f"Found {len(frames)} rendered frames")
        images = [imageio.imread(f) for f in frames[:300]]  # Limit to 300 frames
        imageio.mimwrite('novel_views.mp4', images, fps=30, quality=8)
        print(f"✅ Video saved: novel_views.mp4")
    else:
        print("No PNG frames found in render directory")

In [ ]:
# Display video
from IPython.display import Video
Video('novel_views.mp4', embed=True, width=800)

## 7. Download Results

In [ ]:
# Package results
import shutil
from pathlib import Path

results_dir = Path('results')
results_dir.mkdir(exist_ok=True)

# Copy video
if Path('novel_views.mp4').exists():
    shutil.copy('novel_views.mp4', results_dir)

# Copy final model
model_dirs = list(Path('output/multipleview/my_scene/point_cloud').glob('iteration_*'))
if model_dirs:
    latest = sorted(model_dirs)[-1]
    ply_file = latest / 'point_cloud.ply'
    if ply_file.exists():
        shutil.copy(ply_file, results_dir / '4dgs_model.ply')
        print(f"✅ Model copied: {ply_file.name}")

# Create zip
shutil.make_archive('4dgs_results', 'zip', results_dir)
print(f"\n📦 Results packaged: 4dgs_results.zip")
!ls -lh 4dgs_results.zip

In [ ]:
# Download
from google.colab import files
files.download('4dgs_results.zip')

## 🎉 Done!

Your results include:
- `novel_views.mp4` - Rendered video of novel viewpoints
- `4dgs_model.ply` - 4D Gaussian Splat model

**View your .ply file:**
- SuperSplat: https://playcanvas.com/supersplat/editor
- Luma AI: https://lumalabs.ai/viewer